<a href="https://colab.research.google.com/github/mhamza6000/PIAIC82030-AIC-Q2-Assignments/blob/master/Flower_dectection_assignment_4_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import glob as gb
import cv2
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import RMSprop
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Reading data from Google Drive**

In [ ]:
with tf.device('/device:GPU:1'):
  flower = []
  features = []
  labels = []
  directory = Path('/content/drive/MyDrive/flowers')
  for dir in directory.iterdir():
    flower.append(dir.name)
    print(dir.name)
    for imgpath in dir.iterdir():
      if imgpath.name.endswith('jpg'):
        labels.append(dir.name)
        imgarr = cv2.imread(str(imgpath),cv2.IMREAD_GRAYSCALE)
        imgarr = cv2.resize(imgarr,(150,150))
        features.append(imgarr)

daisy
rose
sunflower
dandelion


In [ ]:
np.array(features).shape

Shuffling Data and Labels with Seed

In [ ]:
seed = np.random.randint(100)
np.random.seed(seed)
np.random.shuffle(features)
np.random.seed(seed)
np.random.shuffle(labels)

# **For One Hot Encode processing Data**

In [ ]:
all_labels = pd.Series(labels)
all_labels.unique()
all_labels_cat = all_labels.astype('category')
all_labels_cat

In [ ]:
all_labels_cat = all_labels.astype('category').cat.codes
print(all_labels_cat.head())
from keras.utils.np_utils import to_categorical
allEncodedLabels = to_categorical(all_labels_cat)
len(allEncodedLabels)

# **Dividing data in Training,validation and Testind datasets**

In [ ]:
data_len = len(features)
train_data = features[:data_len*50//100]
val_data = features[data_len*50//100:data_len*70//100]
test_data = features[data_len*70//100:]
labels_len = len(allEncodedLabels)
train_labels = allEncodedLabels[:labels_len*50//100]
val_labels = allEncodedLabels[labels_len*50//100:labels_len*70//100]
test_labels = allEncodedLabels[labels_len*70//100:]

In [ ]:
train_data = np.array(train_data)
val_data = np.array(val_data)
test_data = np.array(test_data)
# tarin_data = np.array(train_data)
# tarin_data = np.array(train_data)
# tarin_data = np.array(train_data)

# **ReShapeing and Vectorizing data**

In [ ]:
train_data = train_data.reshape((2174,150*150))
val_data = val_data.reshape((869,150*150))
test_data = test_data.reshape((1305,150*150))
train_data = train_data.astype("float32")/255
val_data = val_data.astype("float32")/255
test_data = test_data.astype("float32")/255

# **Preparing Model with Relu as activation function**

In [ ]:
from keras import models
from keras import layers
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(150*150,)))
network.add(layers.Dense(256, activation='relu'))
network.add(layers.Dense(5, activation='softmax'))

In [ ]:
network.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])

In [ ]:
with tf.device('/device:GPU:1'):
  model = network.fit(train_data, train_labels, epochs=100, batch_size=256,validation_data=(val_data,val_labels))

In [ ]:
dict_hist = model.history
accuracy = dict_hist['accuracy']
Val_accuracy = dict_hist['val_accuracy']
Val_loss = dict_hist['val_loss']
loss = dict_hist['loss']

In [ ]:
epoches = np.arange(1,len(dict_hist['accuracy'])+1)
plt.plot(epoches,accuracy,'b',label="Training Accuracy")
plt.plot(epoches,Val_accuracy,'r',label="Validation Accuracy")
plt.title('Training and validation Accuracy')
plt.xlabel("Epoches")
plt.ylabel("Accuracy")
plt.legend()
# plt.show()

In [ ]:
epoches = np.arange(1,len(dict_hist['accuracy']) + 1)
plt.plot(epoches,loss,'b',label="Training loss")
plt.plot(epoches,Val_loss,'r',label="Validation loss")
plt.title('Training and validation loss')
plt.xlabel("Epoches")
plt.ylabel("Loss")
plt.legend()
# plt.show()

# **Evaluation Step**

In [ ]:
test_loss, test_acc = network.evaluate(val_data, val_labels)

# **Prediction**

In [ ]:
predictions = network.predict(test_data)

In [ ]:
y_p = predictions.flatten()
y_t = test_labels.flatten()

In [ ]:
tf.math.confusion_matrix(
    y_p, y_t, num_classes=None, weights=None, dtype=tf.dtypes.int32,
    name=None
)